# EEG Motor Imagery GUI Explained

This notebook is designed to walk you through the Python code for the EEG Motor Imagery Data Collection GUI used for last semester's project. In this notebook we will walk through how the code works, how it aids in collecting EEG data, and ways it can be improved.

By the end of this notebook, you should have a solid understanding of the GUI's functionality, and we'll propose a mini-project for you to collaborate on, focusing on EEG experimental design improvements.

Let's dive in!

---

## Table of Contents

1. [Introduction](#introduction)
2. [Code Overview](#code-overview)
   - [Imports and Initialization](#imports-and-initialization)
   - [Placeholder Functions](#placeholder-functions)
   - [Main Function Structure](#main-function-structure)
   - [Menu System](#menu-system)
   - [Input Handling](#input-handling)
   - [Trial Phases](#trial-phases)
     - [Focus Period](#focus-period)
     - [Arrow Display](#arrow-display)
     - [Loading Bar](#loading-bar)
     - [Rest Period](#rest-period)
   - [Event Handling](#event-handling)
   - [Cleanup](#cleanup)
3. [Understanding the EEG Data Collection](#understanding-the-eeg-data-collection)
4. [Mini-Project: Enhancing the GUI](#mini-project-enhancing-the-gui)
   - [Improvement Suggestions](#improvement-suggestions)
   - [Collaboration Plan](#collaboration-plan)

---

## Introduction

The purpose of this GUI is to facilitate EEG data collection during motor imagery tasks. Users are guided through a series of trials where they are prompted to imagine moving their left or right hand, while EEG data is recorded.

The GUI is designed to be user-friendly and intuitive, providing clear visual cues and a structured sequence to ensure consistent data collection.

---

## What is Motor Imagery?

Motor Imagery is the process of imagining moving a part of your body without actually doing it. This process leads to signals that can be read by an EEG device and classified by machine learning models.

## EEG Data Collection Process

The process of EEG Data collection for two classes of inputs (left arm & right arm), can be explained in 3 steps

1. **Focus Period**
   - This period prompts a cue on screen, usually a plus sign, that prompts the user to begin focusing.
   - This period is either a constant time or a random time.
      - our current implementation has a constant time of 3 seconds.

2. **Recording Period**
   - At the start of this period an arrow appears just above the middle of the screen indicating which arm to think about moving.
      - intuitively a left arrow would indicate the user to think about moving their left are while a right arrow would indicate the user to think about moving their right arm.
   - This period is always of a constant time and a loading bar progresses in its respective direction until it hits the green bar.
      - The green bar is important because it indicates to the user how long they have to continue focusing for.

3. **Rest Period**
   - This period comes directly after the recording period and allows the user to rest for a constant amount of time
   - Commonly, data collected in the previous recording period is stored during this time.

---

## Code Overview

Let's break down the code step by step to understand how it works.
NOTE: if you intend to run the code, please instead run data-collection-gui.py from the same directory.

### Imports and Initialization

In [ ]:
import pygame
import sys
import time
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

- **pygame**: A library for creating graphical user interfaces and handling user input.
- **sys**: Provides access to system-specific parameters and functions.
- **time**: Used for timing events and delays.
- **brainflow**: A library for interfacing with EEG devices. Here, we use it to simulate EEG data collection using a synthetic board.

### Main Function Structure

In [ ]:
# Main():
# Initialize BrainFlow and Pygame
# Set up variables and start the main loop
BoardShim.enable_dev_board_logger()
params = BrainFlowInputParams()
board_id = BoardIds.SYNTHETIC_BOARD.value
board = BoardShim(board_id, params)
board.prepare_session()
board.start_stream()
print("BrainFlow streaming started...")

# Initialize Pygame
pygame.init()
infoObject = pygame.display.Info()
screen = pygame.display.set_mode((infoObject.current_w, infoObject.current_h), pygame.FULLSCREEN)
pygame.display.set_caption("Motor Imagery Task")

# Design the screen
# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Fonts
large_font = pygame.font.SysFont(None, 200)
medium_font = pygame.font.SysFont(None, 100)
small_font = pygame.font.SysFont(None, 50)

# Control Variables
running = True
in_menu = True
in_input = False
in_trial_menu = False
direction = 'left'  # Start with 'left' and alternate
trial_number = 1
total_trials = 10  # Default number of trials

# Bar Settings
green_bar_width = 20
green_bar_height = 200
loading_bar_thickness = 30
arrow_offset_y = 100  # Move arrow up by 100 pixels

# Positions for Green Bars
left_green_bar_pos = (100, infoObject.current_h // 2 - green_bar_height // 2)
right_green_bar_pos = (infoObject.current_w - 100 - green_bar_width, infoObject.current_h // 2 - green_bar_height // 2)

# Center Position
center_pos = (infoObject.current_w // 2, infoObject.current_h // 2)

# Clock
clock = pygame.time.Clock()

# Input Variables
input_text = ""
input_error = False


- The `main` function contains the core structure and components needed to initialize the GUI
    - This includes initializing brainflow and pygame as well creating key structures such as background color and frequently used shapes

### Menu System

The GUI includes two menus:

1. **Main Menu**: Accessed at the start, allows setting the number of trials and starting the experiment.
2. **Trial Menu**: Accessed during trials, allows pausing or quitting the experiment.

#### Main Menu

In [ ]:
def main_menu(screen, large_font, medium_font, small_font, infoObject, total_trials, colors):
    BLACK, WHITE, GREEN, RED = colors
    # Display Main Menu
    screen.fill(BLACK)
    title_text = large_font.render("EEG Motor Imagery", True, WHITE)
    start_text = medium_font.render("Press S to Start", True, GREEN)
    set_text = medium_font.render("Press N to Set Number", True, WHITE)
    quit_text = medium_font.render("Press Q to Quit", True, RED)
    trials_text = small_font.render(f"Total Trials: {total_trials}", True, WHITE)

    # Positioning Text
    title_rect = title_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 4))
    start_rect = start_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 2 - 50))
    set_rect = set_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 2 + 50))
    quit_rect = quit_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 2 + 150))
    trials_rect = trials_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 2 - 150))

    # Blit Text to Screen
    screen.blit(title_text, title_rect)
    screen.blit(start_text, start_rect)
    screen.blit(set_text, set_rect)
    screen.blit(quit_text, quit_rect)
    screen.blit(trials_text, trials_rect)
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_s:
                in_menu = False
            elif event.key == pygame.K_n:
                in_input = True
                in_menu = False
                input_text = ""
                input_error = False
            elif event.key == pygame.K_q:
                running = False

- Displays options to start the experiment, set the number of recordings, or quit.
- Shows the current total number of trials set.

#### Trial Menu

In [ ]:
# Display Trial Menu (Accessible via 'M' during trials)
def trial_menu(screen, medium_font, infoObject, colors):
    BLACK, WHITE, GREEN, RED = colors
    screen.fill(BLACK)
    menu_title = medium_font.render("Trial Menu", True, WHITE)
    quit_text = medium_font.render("Press Q to Quit", True, RED)
    resume_text = medium_font.render("Press R to Resume", True, GREEN)

    # Positioning Text
    menu_title_rect = menu_title.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 3))
    quit_rect = quit_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 2))
    resume_rect = resume_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 2 + 100))

    # Blit Text to Screen
    screen.blit(menu_title, menu_title_rect)
    screen.blit(quit_text, quit_rect)
    screen.blit(resume_text, resume_rect)
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                running = False
            elif event.key == pygame.K_r:
                in_trial_menu = False


- Accessible during trials by pressing 'M'.
- Allows the user to pause the experiment and choose to resume or quit.

### Input Handling

#### Setting Number of Trials

In [ ]:
def set_trials(screen, medium_font, small_font, infoObject, input_text, input_error, colors):
    # Display Input Menu for Setting Number of Trials
    BLACK, WHITE, GREEN, RED = colors
    screen.fill(BLACK)
    prompt_text = medium_font.render("Enter Number of Recordings (Even):", True, WHITE)
    input_display = medium_font.render(input_text, True, GREEN if not input_error else RED)
    instructions_text = small_font.render("Press Enter to Confirm", True, WHITE)

    # Positioning Text
    prompt_rect = prompt_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 3))
    input_rect = input_display.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 2))
    instructions_rect = instructions_text.get_rect(center=(infoObject.current_w // 2, infoObject.current_h // 2 + 100))

    # Blit Text to Screen
    screen.blit(prompt_text, prompt_rect)
    screen.blit(input_display, input_rect)
    screen.blit(instructions_text, instructions_rect)
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                break
            elif event.key == pygame.K_RETURN:
                if input_text.isdigit():
                    entered_number = int(input_text)
                    if entered_number > 0:
                        if entered_number % 2 != 0:
                            entered_number += 1  # Make it even
                            input_error = True
                        total_trials = entered_number
                        in_input = False
                        in_menu = True
                    else:
                        input_error = True
                else:
                    input_error = True
            elif event.key == pygame.K_BACKSPACE:
                input_text = input_text[:-1]
            elif event.unicode.isdigit():
                input_text += event.unicode

### Trial Phases

The experiment consists of several phases within each trial:

1. **Focus Period**
2. **Arrow Display**
3. **Loading Bar**
4. **Rest Period**

Let's explore each phase.

#### Focus Period

In [ ]:
# Draw Focus Period '+' sign
plus_text = large_font.render("+", True, WHITE)
plus_rect = plus_text.get_rect(center=center_pos)
screen.blit(plus_text, plus_rect)
pygame.display.flip()

# Collect data during focus period
focus_duration = 3  # seconds
focus_start_time = time.time()
while time.time() - focus_start_time < focus_duration:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                break
    # Placeholder for data collection during focus
    save_data()
    clock.tick(60)

- A '+' sign appears at the center to prompt the user to focus.
- EEG data is collected during this period using the `save_data()` function.

#### Arrow Display

In [ ]:
arrow_length = 100
arrow_color = WHITE
arrow_width = 20
arrow_y_offset = arrow_offset_y

screen.fill(BLACK)
# draw green bars on either side that indicate when the recording ends
pygame.draw.rect(screen, GREEN, (*left_green_bar_pos, green_bar_width, green_bar_height))
pygame.draw.rect(screen, GREEN, (*right_green_bar_pos, green_bar_width, green_bar_height))

# Draw Arrow
if direction == 'left':
    pygame.draw.polygon(screen, arrow_color, [
        (center_pos[0] - arrow_length, center_pos[1] - arrow_y_offset),
        (center_pos[0], center_pos[1] - arrow_y_offset - arrow_width),
        (center_pos[0], center_pos[1] - arrow_y_offset + arrow_width)
    ])
else:
    pygame.draw.polygon(screen, arrow_color, [
        (center_pos[0] + arrow_length, center_pos[1] - arrow_y_offset),
        (center_pos[0], center_pos[1] - arrow_y_offset - arrow_width),
        (center_pos[0], center_pos[1] - arrow_y_offset + arrow_width)
    ])
pygame.display.flip()

- The loading bar provides a visual cue for how long the user should perform the motor imagery task.
- Moves from the center towards the corresponding green bar (left or right).

#### Loading Bar


In [ ]:
# Loading Bar
loading_duration = 7  # seconds
loading_start_time = time.time()

while time.time() - loading_start_time < loading_duration:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                break
            elif event.key == pygame.K_m:
                in_trial_menu = True
                break

    # Calculate loading bar progress
    elapsed_time = time.time() - loading_start_time
    loading_progress = elapsed_time / loading_duration

    screen.fill(BLACK)
    # Redraw green bars
    pygame.draw.rect(screen, GREEN, (*left_green_bar_pos, green_bar_width, green_bar_height))
    pygame.draw.rect(screen, GREEN, (*right_green_bar_pos, green_bar_width, green_bar_height))
    
    # Redraw Arrow
    if direction == 'left':
        pygame.draw.polygon(screen, arrow_color, [
            (center_pos[0] - arrow_length, center_pos[1] - arrow_y_offset),
            (center_pos[0], center_pos[1] - arrow_y_offset - arrow_width),
            (center_pos[0], center_pos[1] - arrow_y_offset + arrow_width)
        ])
        # Calculate current length of the loading bar
        # From center to left green bar
        max_length = center_pos[0] - (left_green_bar_pos[0] + green_bar_width)
        current_length = loading_progress * max_length

        # Draw loading bar moving left from center
        pygame.draw.rect(screen, WHITE, (
            center_pos[0] - current_length,  # Start at center and move left
            center_pos[1] - loading_bar_thickness // 2,
            current_length,
            loading_bar_thickness
        ))
    else:
        pygame.draw.polygon(screen, arrow_color, [
            (center_pos[0] + arrow_length, center_pos[1] - arrow_y_offset),
            (center_pos[0], center_pos[1] - arrow_y_offset - arrow_width),
            (center_pos[0], center_pos[1] - arrow_y_offset + arrow_width)
        ])
        # Calculate current length of the loading bar
        # From center to right green bar
        max_length = (right_green_bar_pos[0]) - center_pos[0]
        current_length = loading_progress * max_length

        # Draw loading bar moving right from center
        pygame.draw.rect(screen, WHITE, (
            center_pos[0],  # Start at center and move right
            center_pos[1] - loading_bar_thickness // 2,
            current_length,
            loading_bar_thickness
        ))

    pygame.display.flip()

    clock.tick(60)

- The loading bar provides a visual cue for how long the user should perform the motor imagery task.
- Moves from the center towards the corresponding green bar (left or right).

#### Rest Period

In [ ]:
rest_duration = 2  # seconds
rest_start_time = time.time()
while time.time() - rest_start_time < rest_duration:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                break
            elif event.key == pygame.K_m:
                in_trial_menu = True
                break

    # Display Rest Text with Menu Instruction
    screen.fill(BLACK)
    # Redraw green bars
    pygame.draw.rect(screen, GREEN, (*left_green_bar_pos, green_bar_width, green_bar_height))
    pygame.draw.rect(screen, GREEN, (*right_green_bar_pos, green_bar_width, green_bar_height))

    rest_text = small_font.render("Rest (Press M for Menu)", True, WHITE)
    rest_rect = rest_text.get_rect(center=center_pos)
    screen.blit(rest_text, rest_rect)
    pygame.display.flip()
    clock.tick(60)

- Gives the user a short break before the next trial.
- The user can press 'M' to access the trial menu.

### Event Handling

- The GUI handles various user inputs:
  - **'S'**: Start the experiment from the main menu.
  - **'N'**: Set the number of trials.
  - **'Q'**: Quit the application.
  - **'M'**: Access the trial menu during trials.
  - **'R'**: Resume the experiment from the trial menu.
  - **'Escape'**: Exit the application gracefully.

You can locate this implementation in data-collection-gui.py

### Cleanup

In [ ]:
# Cleanup resources
board.stop_stream()
board.release_session()
pygame.quit()
sys.exit()

- Stops the EEG data stream and releases resources.
- Closes the Pygame window and exits the application.

---

## Understanding the EEG Data Collection

While the current code uses a synthetic EEG board for simulation, it is structured to integrate actual EEG data collection seamlessly.

### Data Collection Points

Although not implemented in this code, a 'save_data()' function would be called at the end of each recording and executed during the focus period.

## Integrating Real EEG Devices

To use a real EEG device:

- Configure `BrainFlowInputParams` with your device's connection parameters.
- Implement the `save_data()` function to record and store EEG data during the trials.


---

## Mini-Project: Enhancing the GUI

Now that you're familiar with the code, let's collaborate on improving the GUI with a focus on EEG experimental design.

### Improvement Suggestions

1. **Introduce Randomized Inter-Trial Intervals (ITI)**:
   - Add variability to the rest periods between trials to prevent anticipatory effects.
   - Randomize the duration within a specified range (e.g., 2-5 seconds).

2. **Include a Calibration Phase**:
   - Before the main experiment, include a calibration phase to record baseline EEG data.
   - Helps in adjusting for individual differences and improving data accuracy.

3. **General Aesthetic Enhancements**:
   - Implement any Aesthetic changes you think would improve the user experience

4. **User Experience Enhancements**:
   - Implement any functional changes you think would improve the user experience